<a href="https://colab.research.google.com/github/hruthiksiva/DataScience-Competitions/blob/main/enigma_round2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle API

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c consumer-complaints

100% 159M/159M [00:00<00:00, 148MB/s]
100% 159M/159M [00:00<00:00, 172MB/s]
  0% 0.00/1.81M [00:00<?, ?B/s]
100% 1.81M/1.81M [00:00<00:00, 121MB/s]
100% 70.0M/70.0M [00:00<00:00, 77.8MB/s]



In [3]:
!unzip \*.zip  && rm *.zip

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  SAMPLE_SUBMISSION.csv.zip
  inflating: SAMPLE_SUBMISSION.csv   

3 archives were successfully processed.


#Imports

In [36]:
import pandas as pd
import numpy as np

# Data Preprocessing

In [6]:
df=pd.read_csv('train.csv')

In [8]:
#df.info()
#df.head()
#df.describe()
#df.isnull().sum()

In [19]:
df.head()

,Unnamed: 0,Complaint_ID,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Consumer disputed?,Timely response?
0,0,2535701,6/8/2017,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,NaN,NaN,"Caliber Home Loans, Inc.",FL,34744,NaN,NaN,Fax,6/12/2017,Closed with explanation,NaN,1
1,1,3314889,7/22/2019,Credit card or prepaid card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,On XX/XX/2019 all of my credit and debit cards...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,CA,900XX,NaN,Consent provided,Web,7/22/2019,Closed with explanation,NaN,1
2,2,3612478,4/18/2020,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I was made aware of a negative account and inq...,NaN,"EQUIFAX, INC.",CA,945XX,NaN,Consent provided,Web,4/18/2020,Closed with explanation,NaN,1
3,3,2069704,8/18/2016,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",TX,77014,NaN,NaN,Phone,8/19/2016,Closed with monetary relief,No,1
4,4,3354927,8/27/2019,Debt collection,Auto debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,Bridgecrest Acceptance Corporation,PA,19143,Servicemember,Consent not provided,Web,8/27/2019,Closed with explanation,NaN,1


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Unnamed: 0                    1048575 non-null  int64 
 1   Complaint_ID                  1048575 non-null  int64 
 2   Date received                 1048575 non-null  object
 3   Product                       1048575 non-null  object
 4   Sub-product                   947674 non-null   object
 5   Issue                         1048575 non-null  object
 6   Sub-issue                     766056 non-null   object
 7   Consumer complaint narrative  310809 non-null   object
 8   Company public response       446648 non-null   object
 9   Company                       1048575 non-null  object
 10  State                         1027557 non-null  object
 11  ZIP code                      954841 non-null   object
 12  Tags                          129277 non-n

### Encoding

In [53]:
from sklearn.preprocessing import LabelEncoder
listing=['Timely response?','Consumer disputed?','Company response to consumer',
         'Consumer consent provided?','Submitted via','Tags','State','Company public response',
         #'Issue','Sub-issue',
         'Product','Sub-product']
for i in listing:
  lb=LabelEncoder()
  df[str(i)]= lb.fit_transform(df[str(i)])

TypeError: ignored

In [ ]:
print(df['Sub-product'].describe())

In [57]:
df['Sub-issue'].unique()

array([nan,
       "Credit card company isn't resolving a dispute about a purchase on your statement",
       'Information belongs to someone else',
       'Threatened or suggested your credit would be damaged',
       'Their investigation did not fix an error on your report',
       'Account status incorrect',
       'Threatened to sue you for very old debt',
       'Investigation took more than 30 days',
       'Account information incorrect',
       'Problem with personal statement of dispute',
       'Difficulty submitting a dispute or getting information about a dispute over the phone',
       'Attempted to collect wrong amount',
       'Trouble with how payments are being handled', 'Account terms',
       'Information is not mine', 'Account status', 'Debt is not yours',
       'Was not notified of investigation status or results',
       'Debt was paid', 'Contacted you after you asked them to stop',
       'Public record', 'Old information reappears or never goes away',
       'S

# Data Visualization

## Uniqueness

In [35]:
#print(df['Consumer disputed?'].unique())
#print(df['Company response to consumer'].unique())
print(df['Date received'].unique())


['6/8/2017' '7/22/2019' '4/18/2020' ... '9/8/2013' '6/16/2018' '3/9/2014']


In [55]:
df.isnull().sum()

Unnamed: 0                           0
Complaint_ID                         0
Date received                        0
Product                              0
Sub-product                     100901
Issue                                0
Sub-issue                       282519
Consumer complaint narrative    737766
Company public response         601927
Company                              0
State                            21018
ZIP code                         93734
Tags                            919298
Consumer consent provided?      367626
Submitted via                        0
Date sent to company                 0
Company response to consumer         1
Consumer disputed?              719432
Timely response?                     0
dtype: int64

##Correlation


In [ ]:
print(df['']. corr(df['']))

# Train Test Split

In [18]:
y = df.loc[:,'Timely response?']
X = df.drop(['Complaint_ID','Timely response?','Unnamed: 0'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Models

# Prediction

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')
#

# Submission

In [ ]:
tdf=pd.read_csv('test.csv')

## Submission File